# Importing Header and SoilPrep

In [1]:
import import_ipynb 
from Header import *

importing Jupyter notebook from Header.ipynb


In [2]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


## Loading Data and MetaData

In [3]:
import import_ipynb
from LoadDataMetaData import *

importing Jupyter notebook from LoadDataMetaData.ipynb


In [4]:
# with open ('meta_data.pickle', 'rb') as file:
#     MetaData = pickle.load(file)

# sg_filters = MetaData['sg_filters']
# window_lengths = MetaData['window_lengths']
# prepare_spec = MetaData['prepare_spec']
# nbands_sampling = MetaData['nbands_sampling']
# target_names = MetaData['target_names']
# prepare_target = MetaData['prepare_target']
# ml_methods = MetaData['ml_methods']
# clr = MetaData['clr']


# with open ('data.pickle', 'rb') as file:
#     Data = pickle.load(file)

# spectra = Data['spectra']
# T = Data['T']
# NT = Data['NT']
# spec1 = Data['spec1'] 
# spec2 = Data['spec2']
# smth_spec = Data['smth_spec']
# fod_spec = Data['fod_spec']
# cr_spec = Data['cr_spec'] 
# log_spec = Data['log_spec']
# sampled_spec = Data['sampled_spec']
# sampled_cr = Data['sampled_cr']

# fod_sampled= Data['fod_sampled']
# sampled_log = Data['sampled_log'] 
# rand_t = Data['rand_t']
# rand_nt = Data['rand_nt'] 

# Building Model Tree (Mtree)

In [5]:
import os
os.system('say "your Meta Tree started building"')
import time

In [6]:
# 0. Available machine learning regression models --------------------------------- (7)
#ml_methods = ['mult', 'plsr', 'randomforest', 'svr', 'ridge', 'gbrt']
#ml_methods = ['mult', 'plsr', 'cubist', 'randomforest', 'ridge' 'gbrt', 'svr']

In [7]:

def build_tree_for (method_name):
    tree ={}
    start = time.time()
    m = method_name
    #-- code to build tree----
    for t in target_names:
        print('tree for: '+ m +' ------> running on: ' + t)
        tree[t] ={}
        for tp in prepare_target:
            tree[t][tp] ={}
            for p in prepare_spec:
                tree[t][tp][p] ={}
                for n in nbands_sampling:
                    tree[t][tp][p][n] ={}
                    Y = tree[t][tp][p][n]
                    
                        
                    #------ setting spec to appropriate (sampled) spectra----
                    if p == 'none':
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_spec[100]
                            else:
                                spec = spec2[51]
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_spec[n]
                        
                    elif p == 'fod':
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_fod[100]
                            else:
                                spec = fod_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_fod[n]
                    elif p == 'fod2':
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = fod_sampled[100]
                            else:
                                spec = fod_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = fod_sampled[n]            
                    elif p== 'continuum':  
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_cr[100]
                            else:
                                spec = cr_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_cr[n]
                        
                    else: 
                        if n == 0:
                            if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
                                #---- reched here due to tree based methods e.g. randomforest--
                                spec = sampled_log[100]
                            else:
                                spec = log_spec
                        else:
                            #---- reached here with some n!=0----
                            spec = sampled_log[n]
                    
                    
                    
                    #---- target selection and normalization ---
                    if tp == 'none':
                        y = T[target_names.index(t)]
                        rand_n = rand_t[target_names.index(t)]  #-- for future use in train-test split
                    else:
                        y = NT[target_names.index(t)]
                        rand_n = rand_nt[target_names.index(t)] #-- for future use in train-test split
                        #print('one more target set')
                            
                    #---- performing train-test split----------------------
                    X_train, X_test, y_train, y_test = train_test_split(spec, y, test_size= tst_siz, random_state=rand_n)
                        
                        
                    #------INITIATING the appropriate models-----------------------------------------------
                    #----- PLSR: best parameters, fitting, and prediction
                    if m == 'plsr':
                        n_com = best_param_PLSR (X_train, y_train, rand_n, max_n_comp)
                        Y['n_comp'] = n_com
                        Model = PLSRegression(n_components=n_com, scale=True)
                    #----- MULTILINEAR:  fitting, and prediction---------- 
                    elif m == 'mult':
                        Model = linear_model.LinearRegression()    
                    #----- RANDOM_FOREST:   fitting, and prediction---------- 
                    elif m == 'randomforest': 
                        Model = RandomForestRegressor(random_state= 23)    
                    #----- CUBIST REGRESSION:    fitting and prediction---------
                    elif m == 'cubist':
                        Model = Cubist(n_rules = 50, n_committees = 5, random_state = 42)    
                    #------ SUPPORT VECTOR MACHINE FOR REGRESSION: fitting and prediction-----------      
                    elif m == 'svr': 
                        Model = SVR()
                    #------ RIDGE REGRESSION: fitting and prediction-----------      
                    elif m == 'ridge': 
                        Model = KernelRidge()
                    #------ LASSO REGRESSION: fitting and prediction-----------     
                    elif m == 'lasso': 
                        Model = Lasso(alpha=10)    
                    #------ GRADIENT BOOSTING REGRESSION: fitting and prediction-----------      
                    else: 
                        Model = GradientBoostingRegressor()                         
                        
                    Model.fit(X_train, y_train)
                    y_pred = Model.predict(X_test)
                    yhat_pred = Model.predict(X_train)
                                
                    Y['test'] = y_test
                    Y['testP'] = y_pred
                    Y['train'] = y_train
                    Y['trainP'] = yhat_pred
                    Y['iqrp_test'] = find_iqrp(y_pred, y_test)
                    Y['r2_test'] = find_r2(y_pred, y_test)
                    Y['rpd_test'] = find_rpd(y_pred, y_test)
                    Y['rmse_test'] = find_rmse(y_pred, y_test)
                    
                    Y['r2_train'] = find_r2(yhat_pred, y_train)
                                                                
                                                                         
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return (tree.copy())


## Loading the saved Mtree

In [8]:
# with open ('Mtree.pickle', 'rb') as file:
#     Mtree = pickle.load(file)

## Mtree Building (run below code only once)

In [9]:
#max_n_comp = 2
tst_siz = 0.20

In [10]:
Mtree ={}

In [11]:
Mtree['lasso'] = build_tree_for ('lasso')

tree for: lasso ------> running on: Sand
tree for: lasso ------> running on: Silt
tree for: lasso ------> running on: Clay
tree for: lasso ------> running on: TOC
tree for: lasso ------> running on: CaCO3
End time - Start time = 6.830242156982422


In [12]:
Mtree['mult'] = build_tree_for ('mult')
Mtree['svr'] = build_tree_for ('svr')


Mtree['cubist'] = build_tree_for ('cubist')
Mtree['randomforest'] = build_tree_for ('randomforest')
Mtree['gbrt'] = build_tree_for ('gbrt')

tree for: mult ------> running on: Sand
tree for: mult ------> running on: Silt
tree for: mult ------> running on: Clay
tree for: mult ------> running on: TOC
tree for: mult ------> running on: CaCO3
End time - Start time = 6.107760429382324
tree for: svr ------> running on: Sand
tree for: svr ------> running on: Silt
tree for: svr ------> running on: Clay
tree for: svr ------> running on: TOC
tree for: svr ------> running on: CaCO3
End time - Start time = 6.266808032989502
tree for: cubist ------> running on: Sand
tree for: cubist ------> running on: Silt
tree for: cubist ------> running on: Clay
tree for: cubist ------> running on: TOC
tree for: cubist ------> running on: CaCO3
End time - Start time = 97.27724099159241
tree for: randomforest ------> running on: Sand
tree for: randomforest ------> running on: Silt
tree for: randomforest ------> running on: Clay
tree for: randomforest ------> running on: TOC
tree for: randomforest ------> running on: CaCO3
End time - Start time = 125.1

In [13]:
max_n_comp = 10
Mtree['plsr'] = build_tree_for ('plsr')

tree for: plsr ------> running on: Sand


C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

...

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(3) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(4) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(5) should be in [1, n_features].n_components=2 will be used instead. In version 1.1 (renaming of 0.26), an error will be raised.
  warnings.warn(
C:\Users\abhis\anaconda3\lib\site-packages\sklearn\cross_decomposition\_pls.py:230: FutureWarning: As of version 0.24, n_components(6) should be in [1, n_features].n_

End time - Start time = 38.1894896030426


In [14]:
Mtree['ridge'] = build_tree_for ('ridge')

tree for: ridge ------> running on: Sand
tree for: ridge ------> running on: Silt
tree for: ridge ------> running on: Clay
tree for: ridge ------> running on: TOC
tree for: ridge ------> running on: CaCO3
End time - Start time = 5.283473253250122


## Saving the Mtree

In [15]:
with open ('Mtree.pickle', 'wb') as file:
    pickle.dump(Mtree, file)

## Best of all worlds

In [16]:
def best_model_parameters (Mtree, target, method, scorer):
    t=target
    m= method
    
    best_score = -1
    best_n_comp = 'NA'
    
    
    for tp in prepare_target:
        for p in prepare_spec:
            for n in nbands_sampling:
                Y = Mtree[m][t][tp][p][n]
                r2_train = Y['r2_train']
                r2_test = Y['r2_test']
                    
                if scorer == 'iqrp':
                    cur_score = Y['iqrp_test']
                elif scorer == 'rpd':
                    cur_score = Y['rpd_test']
                else:
                    cur_score = Y['r2_test']  
                    
                if cur_score > best_score and r2_train >= r2_test:
                    best_score = cur_score
                    best_tp = tp
                    best_n = n
                    best_p = p
                    if m == 'plsr':
                        best_n_comp = Y['n_comp']
                            
    param_list = [scorer, np.round(best_score,2), 'Spec:', best_p, 'bands:', best_n, 'Tar:', best_tp]                                 
    return (param_list)                                
    

In [17]:
def best_score_for (Mtree, target, scorer):
    
    for method in ml_methods:
        param_list= best_model_parameters (Mtree, target, method, scorer)
        print('For:'+target+'->', param_list, ':'+method)
     
    return

In [18]:
best_score_for(Mtree, 'Sand', 'iqrp')

For:Sand-> ['iqrp', 2.81, 'Spec:', 'none', 'bands:', 9, 'Tar:', 'minmax'] :mult
For:Sand-> ['iqrp', 3.14, 'Spec:', 'none', 'bands:', 39, 'Tar:', 'minmax'] :cubist
For:Sand-> ['iqrp', 2.1, 'Spec:', 'log', 'bands:', 35, 'Tar:', 'minmax'] :svr
For:Sand-> ['iqrp', 2.06, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :ridge
For:Sand-> ['iqrp', 1.7, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :lasso
For:Sand-> ['iqrp', 2.98, 'Spec:', 'fod2', 'bands:', 30, 'Tar:', 'minmax'] :plsr
For:Sand-> ['iqrp', 2.86, 'Spec:', 'fod2', 'bands:', 25, 'Tar:', 'minmax'] :randomforest
For:Sand-> ['iqrp', 2.93, 'Spec:', 'fod2', 'bands:', 20, 'Tar:', 'minmax'] :gbrt


In [19]:
best_score_for(Mtree, 'Sand', 'r2')

For:Sand-> ['r2', 0.63, 'Spec:', 'none', 'bands:', 9, 'Tar:', 'minmax'] :mult
For:Sand-> ['r2', 0.71, 'Spec:', 'none', 'bands:', 39, 'Tar:', 'minmax'] :cubist
For:Sand-> ['r2', 0.34, 'Spec:', 'log', 'bands:', 35, 'Tar:', 'minmax'] :svr
For:Sand-> ['r2', 0.31, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :ridge
For:Sand-> ['r2', -0.01, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'none'] :lasso
For:Sand-> ['r2', 0.67, 'Spec:', 'fod2', 'bands:', 30, 'Tar:', 'minmax'] :plsr
For:Sand-> ['r2', 0.65, 'Spec:', 'fod2', 'bands:', 25, 'Tar:', 'minmax'] :randomforest
For:Sand-> ['r2', 0.66, 'Spec:', 'fod2', 'bands:', 20, 'Tar:', 'minmax'] :gbrt


In [20]:
best_score_for(Mtree, 'Silt', 'iqrp')

For:Silt-> ['iqrp', 1.66, 'Spec:', 'fod2', 'bands:', 7, 'Tar:', 'minmax'] :mult
For:Silt-> ['iqrp', 2.0, 'Spec:', 'fod2', 'bands:', 27, 'Tar:', 'none'] :cubist
For:Silt-> ['iqrp', 1.58, 'Spec:', 'continuum', 'bands:', 11, 'Tar:', 'minmax'] :svr
For:Silt-> ['iqrp', 1.72, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'none'] :ridge
For:Silt-> ['iqrp', 1.45, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :lasso
For:Silt-> ['iqrp', 1.78, 'Spec:', 'none', 'bands:', 10, 'Tar:', 'none'] :plsr
For:Silt-> ['iqrp', 1.92, 'Spec:', 'fod2', 'bands:', 11, 'Tar:', 'minmax'] :randomforest
For:Silt-> ['iqrp', 1.91, 'Spec:', 'fod2', 'bands:', 10, 'Tar:', 'none'] :gbrt


In [21]:
best_score_for(Mtree, 'Silt', 'r2')

For:Silt-> ['r2', 0.24, 'Spec:', 'fod2', 'bands:', 7, 'Tar:', 'none'] :mult
For:Silt-> ['r2', 0.47, 'Spec:', 'fod2', 'bands:', 27, 'Tar:', 'none'] :cubist
For:Silt-> ['r2', 0.16, 'Spec:', 'continuum', 'bands:', 11, 'Tar:', 'minmax'] :svr
For:Silt-> ['r2', 0.29, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'none'] :ridge
For:Silt-> ['r2', -0.0, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'none'] :lasso
For:Silt-> ['r2', 0.34, 'Spec:', 'none', 'bands:', 10, 'Tar:', 'none'] :plsr
For:Silt-> ['r2', 0.43, 'Spec:', 'fod2', 'bands:', 11, 'Tar:', 'minmax'] :randomforest
For:Silt-> ['r2', 0.43, 'Spec:', 'fod2', 'bands:', 10, 'Tar:', 'none'] :gbrt


In [22]:
best_score_for(Mtree, 'Clay', 'iqrp')

For:Clay-> ['iqrp', 2.99, 'Spec:', 'log', 'bands:', 15, 'Tar:', 'minmax'] :mult
For:Clay-> ['iqrp', 2.7, 'Spec:', 'fod2', 'bands:', 27, 'Tar:', 'none'] :cubist
For:Clay-> ['iqrp', 1.95, 'Spec:', 'log', 'bands:', 90, 'Tar:', 'minmax'] :svr
For:Clay-> ['iqrp', 1.98, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'minmax'] :ridge
For:Clay-> ['iqrp', 1.43, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'none'] :lasso
For:Clay-> ['iqrp', 2.24, 'Spec:', 'continuum', 'bands:', 21, 'Tar:', 'minmax'] :plsr
For:Clay-> ['iqrp', 2.37, 'Spec:', 'continuum', 'bands:', 23, 'Tar:', 'minmax'] :randomforest
For:Clay-> ['iqrp', 3.04, 'Spec:', 'continuum', 'bands:', 23, 'Tar:', 'minmax'] :gbrt


In [23]:
best_score_for(Mtree, 'Clay', 'r2')

For:Clay-> ['r2', 0.77, 'Spec:', 'log', 'bands:', 15, 'Tar:', 'minmax'] :mult
For:Clay-> ['r2', 0.72, 'Spec:', 'fod2', 'bands:', 27, 'Tar:', 'none'] :cubist
For:Clay-> ['r2', 0.46, 'Spec:', 'log', 'bands:', 90, 'Tar:', 'minmax'] :svr
For:Clay-> ['r2', 0.48, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'minmax'] :ridge
For:Clay-> ['r2', -0.01, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :lasso
For:Clay-> ['r2', 0.59, 'Spec:', 'continuum', 'bands:', 21, 'Tar:', 'minmax'] :plsr
For:Clay-> ['r2', 0.64, 'Spec:', 'continuum', 'bands:', 23, 'Tar:', 'minmax'] :randomforest
For:Clay-> ['r2', 0.78, 'Spec:', 'continuum', 'bands:', 23, 'Tar:', 'minmax'] :gbrt


In [24]:
best_score_for(Mtree, 'TOC', 'iqrp')

For:TOC-> ['iqrp', 1.74, 'Spec:', 'log', 'bands:', 20, 'Tar:', 'minmax'] :mult
For:TOC-> ['iqrp', 2.21, 'Spec:', 'log', 'bands:', 35, 'Tar:', 'none'] :cubist
For:TOC-> ['iqrp', 1.36, 'Spec:', 'log', 'bands:', 10, 'Tar:', 'none'] :svr
For:TOC-> ['iqrp', 1.65, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'none'] :ridge
For:TOC-> ['iqrp', 1.16, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'none'] :lasso
For:TOC-> ['iqrp', 1.76, 'Spec:', 'log', 'bands:', 20, 'Tar:', 'minmax'] :plsr
For:TOC-> ['iqrp', 2.29, 'Spec:', 'continuum', 'bands:', 33, 'Tar:', 'none'] :randomforest
For:TOC-> ['iqrp', 2.49, 'Spec:', 'fod2', 'bands:', 40, 'Tar:', 'minmax'] :gbrt


In [25]:
best_score_for(Mtree, 'TOC', 'r2')

For:TOC-> ['r2', 0.6, 'Spec:', 'log', 'bands:', 20, 'Tar:', 'minmax'] :mult
For:TOC-> ['r2', 0.69, 'Spec:', 'log', 'bands:', 35, 'Tar:', 'none'] :cubist
For:TOC-> ['r2', 0.23, 'Spec:', 'fod2', 'bands:', 19, 'Tar:', 'minmax'] :svr
For:TOC-> ['r2', 0.45, 'Spec:', 'log', 'bands:', 100, 'Tar:', 'minmax'] :ridge
For:TOC-> ['r2', -0.01, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :lasso
For:TOC-> ['r2', 0.61, 'Spec:', 'log', 'bands:', 20, 'Tar:', 'minmax'] :plsr
For:TOC-> ['r2', 0.72, 'Spec:', 'continuum', 'bands:', 33, 'Tar:', 'none'] :randomforest
For:TOC-> ['r2', 0.81, 'Spec:', 'fod2', 'bands:', 40, 'Tar:', 'minmax'] :gbrt


In [26]:
best_score_for(Mtree, 'CaCO3', 'iqrp')

For:CaCO3-> ['iqrp', 2.1, 'Spec:', 'log', 'bands:', 5, 'Tar:', 'none'] :mult
For:CaCO3-> ['iqrp', 2.12, 'Spec:', 'log', 'bands:', 5, 'Tar:', 'none'] :cubist
For:CaCO3-> ['iqrp', 1.9, 'Spec:', 'continuum', 'bands:', 29, 'Tar:', 'minmax'] :svr
For:CaCO3-> ['iqrp', 2.12, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'none'] :ridge
For:CaCO3-> ['iqrp', 1.46, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'minmax'] :lasso
For:CaCO3-> ['iqrp', 2.16, 'Spec:', 'continuum', 'bands:', 15, 'Tar:', 'none'] :plsr
For:CaCO3-> ['iqrp', 2.15, 'Spec:', 'fod2', 'bands:', 5, 'Tar:', 'none'] :randomforest
For:CaCO3-> ['iqrp', 2.29, 'Spec:', 'none', 'bands:', 3, 'Tar:', 'none'] :gbrt


In [27]:
best_score_for(Mtree, 'CaCO3', 'r2')

For:CaCO3-> ['r2', 0.51, 'Spec:', 'log', 'bands:', 5, 'Tar:', 'none'] :mult
For:CaCO3-> ['r2', 0.52, 'Spec:', 'log', 'bands:', 5, 'Tar:', 'none'] :cubist
For:CaCO3-> ['r2', 0.41, 'Spec:', 'continuum', 'bands:', 29, 'Tar:', 'minmax'] :svr
For:CaCO3-> ['r2', 0.52, 'Spec:', 'log', 'bands:', 0, 'Tar:', 'none'] :ridge
For:CaCO3-> ['r2', -0.0, 'Spec:', 'none', 'bands:', 0, 'Tar:', 'none'] :lasso
For:CaCO3-> ['r2', 0.54, 'Spec:', 'continuum', 'bands:', 15, 'Tar:', 'none'] :plsr
For:CaCO3-> ['r2', 0.54, 'Spec:', 'fod2', 'bands:', 5, 'Tar:', 'none'] :randomforest
For:CaCO3-> ['r2', 0.59, 'Spec:', 'none', 'bands:', 3, 'Tar:', 'none'] :gbrt


## Plotting Model Accuracy (ipywidgets)

In [28]:
def plot_model_acc (method, target, target_preprocessing, spec_preprocessing, n_bands):
    
    m = method
    t = target
    
    i = target_names.index(target)
    
    
    p = spec_preprocessing
    n = n_bands
    tp = target_preprocessing
    
    Y = Mtree[m][t][tp][p][n]
    
    y_test = Y['test']
    y_pred = Y['testP']
    y_train = Y['train']
    yhat_pred = Y['trainP']
    
    if m == 'plsr':
        n_com = Y['n_comp']
        y_pred = y_pred[:,0]
        yhat_pred = yhat_pred[:,0]
    
    
    iqrp_test = Y['iqrp_test']
    r2_test = Y['r2_test']
    rpd_test = Y['rpd_test']
    
    iqrp_train = find_iqrp(yhat_pred, y_train)
    r2_train = find_r2(yhat_pred, y_train)
    rpd_train = find_rpd(yhat_pred, y_train)
    
    y_tp = pd.DataFrame({'actual':y_test.values, 'predic': y_pred})
    z = np.polyfit(y_test, y_pred, 1)
    
    yhat_tp = pd.DataFrame({'actual':y_train.values, 'predic': yhat_pred})
    zhat = np.polyfit(y_train, yhat_pred, 1)
    
    fig, axes = plt.subplots(1,2, figsize=(18,8))
    
    #with plt.style.context(('ggplot')): ---- PLOT of test-prediction --------------------------------------
    y_tp.plot.scatter(ax= axes[0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0].plot(y_test, np.polyval(z, y_test),  c='blue', linewidth=1)
    axes[0].plot(y_test, y_test, color='green', linewidth=1)
    axes[0].tick_params(axis='both', labelsize=10)
    axes[0].text(0.05, 0.95, target_names[i]+' (Test Data)', transform=axes[0].transAxes, fontsize = 20, color = clr[i])
    axes[0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(iqrp_test), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.05, 0.85, 'RPD ={:.2f}'.format(rpd_test), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(r2_test,3)), transform=axes[0].transAxes, fontsize = 16)
    axes[0].text(0.95, 0.15, 'Method: '+method, transform=axes[0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    
    if method == 'plsr':
        axes[0].text(0.95, 0.05, 'n_component={:.2f}'.format(n_com), transform=axes[0].transAxes, 
                    horizontalalignment='right', fontsize = 12)
    
    #---------------------------------- ---- PLOT of train-prediction --------------------------------------
    yhat_tp.plot.scatter(ax= axes[1], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[1].plot(y_train, np.polyval(zhat, y_train),  c='blue', linewidth=1)
    axes[1].plot(y_train, y_train, color='green', linewidth=1)
    axes[1].tick_params(axis='both', labelsize=10)
    axes[1].text(0.05, 0.95,  target_names[i]+' (Training Data)', transform=axes[1].transAxes,fontsize = 20, color = clr[i])
    axes[1].text(0.05, 0.90, 'IQRP ={:.2f}'.format(iqrp_train), transform=axes[1].transAxes, fontsize = 16)
    axes[1].text(0.05, 0.85, 'RPD ={:.2f}'.format(rpd_train), transform=axes[1].transAxes, fontsize = 16)
    axes[1].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(r2_train,3)), transform=axes[1].transAxes, fontsize = 16)
    axes[1].text(0.95, 0.15, 'Method: '+method, transform=axes[1].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    return    

In [29]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target, n_bands = nbands_sampling )

interactive(children=(Dropdown(description='method', options=('mult', 'cubist', 'svr', 'ridge', 'lasso', 'plsrâ€¦

<function __main__.plot_model_acc(method, target, target_preprocessing, spec_preprocessing, n_bands)>